In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from yahoo_fin import stock_info as si
from collections import deque

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import random

import multiprocessing
    
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
    
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
tf.keras.mixed_precision.experimental.set_policy(policy) 

In [2]:
def create_model(sequence_length, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop", bidirectional=False,layer_activation="linear"):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            # first layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True), input_shape=(None, sequence_length)))
            else:
                model.add(cell(units, return_sequences=True, input_shape=(None, sequence_length)))
        elif i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(1, activation=layer_activation))
    model.compile(loss=loss, metrics=["mean_absolute_error"], optimizer=optimizer)
    return model

In [4]:
#def run_tensorflow():

# create these folders if they does not exist
# Window size or the sequence length
N_STEPS = 152
# Lookup step, 1 is the next day
#LOOKUP_STEP = int(run_dict[run]["LOOKUP_STEP"])

# test ratio size, 0.2 is 20%
TEST_SIZE = 0.3
# features to use
FEATURE_COLUMNS = ["close_0","ema_0","high_0","low_0","open_0","rsi_0","sma_0","volume_0","close_1","ema_1","high_1","low_1","open_1","rsi_1","sma_1","volume_1",
                   "close_2","ema_2","high_2","low_2","open_2","rsi_2","sma_2","volume_2","close_3","ema_3","high_3","low_3","open_3","rsi_3","sma_3","volume_3",
                   "close_4","ema_4","high_4","low_4","open_4","rsi_4","sma_4","volume_4","close_5","ema_5","high_5","low_5","open_5","rsi_5","sma_5","volume_5",
                   "close_6","ema_6","high_6","low_6","open_6","rsi_6","sma_6","volume_6","close_7","ema_7","high_7","low_7","open_7","rsi_7","sma_7","volume_7",
                   "close_8","ema_8","high_8","low_8","open_8","rsi_8","sma_8","volume_8","close_9","ema_9","high_9","low_9","open_9","rsi_9","sma_9","volume_9",
                   "close_10","ema_10","high_10","low_10","open_10","rsi_10","sma_10","volume_10","close_11","ema_11","high_11","low_11","open_11","rsi_11","sma_11","volume_11",
                   "close_12","ema_12","high_12","low_12","open_12","rsi_12","sma_12","volume_12","close_13","ema_13","high_13","low_13","open_13","rsi_13","sma_13","volume_13",
                   "close_14","ema_14","high_14","low_14","open_14","rsi_14","sma_14","volume_14","close_15","ema_15","high_15","low_15","open_15","rsi_15","sma_15","volume_15",
                   "close_16","ema_16","high_16","low_16","open_16","rsi_16","sma_16","volume_16","close_17","ema_17","high_17","low_17","open_17","rsi_17","sma_17","volume_17",
                   "close_18","ema_18","high_18","low_18","open_18","rsi_18","sma_18","volume_18"]
TARGET_COLUMNS = ["close_19","high_19","low_19","open_19"]
# date now
date_now = time.strftime("%Y-%m-%d")

### model parameters

N_LAYERS = 3
# LSTM cell
CELL = LSTM
# 256 LSTM neurons
UNITS = 1000
# 40% dropout
DROPOUT = 0.25
# whether to use bidirectional RNNs
BIDIRECTIONAL = True

### training parameters

# mean absolute error loss
# LOSS = "mae"
# huber loss
LOSS = "huber_loss"
OPTIMIZER = "adam"
BATCH_SIZE = 64
EPOCHS = 50

LAYER_ACTIVATION = "linear"

# Stock market
ticker = "MIXED"
ticker_data_filename = os.path.join("data", f"{ticker}_{date_now}.csv")
# model name to save, making it as unique as possible based on parameters
model_name = f"{date_now}_{ticker}-{LOSS}-{OPTIMIZER}-{CELL.__name__}-{LAYER_ACTIVATION}-layers-{N_LAYERS}-units-{UNITS}"
if BIDIRECTIONAL:
    model_name += "-b"

#----------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------#

#try:
if not os.path.isdir("results"):
    os.mkdir("results")

if not os.path.isdir("logs"):
    os.mkdir("logs")

if not os.path.isdir("data"):
    os.mkdir("data")

# load the data
data = pd.read_csv('../data/processed/all_processed_20.csv')

# construct the model
model = create_model(N_STEPS, loss=LOSS, units=UNITS, cell=CELL, n_layers=N_LAYERS,
                    dropout=DROPOUT, optimizer=OPTIMIZER, bidirectional=BIDIRECTIONAL, layer_activation=LAYER_ACTIVATION)

# some tensorflow callbacks
checkpointer = ModelCheckpoint(os.path.join("results", model_name + ".h5"), save_weights_only=True, save_best_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))

X = data[FEATURE_COLUMNS]
y = data[TARGET_COLUMNS]

# convert to numpy arrays
X = np.array(X)
y = np.array(y)

# reshape X to fit the neural network
X = X.reshape((X.shape[0], 1, X.shape[1]))

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True)

history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpointer, tensorboard],
                    verbose=1)

model.save(os.path.join("results", model_name) + ".h5")

#except:
#    print("There was an attempt.")
tf.keras.backend.clear_session()

Train on 67191 samples, validate on 28797 samples
Epoch 1/50
67136/67191 [============================>.] - ETA: 0s - loss: 0.4501 - mean_absolute_error: 0.8125
Epoch 00001: val_loss improved from inf to 0.44328, saving model to results\2021-12-02_MIXED-huber_loss-adam-LSTM-linear-layers-3-units-1000-b.h5
67191/67191 [==============================] - 42s 632us/sample - loss: 0.4501 - mean_absolute_error: 0.8125 - val_loss: 0.4433 - val_mean_absolute_error: 0.8046
Epoch 2/50
67136/67191 [============================>.] - ETA: 0s - loss: 0.4340 - mean_absolute_error: 0.7927
Epoch 00002: val_loss improved from 0.44328 to 0.43512, saving model to results\2021-12-02_MIXED-huber_loss-adam-LSTM-linear-layers-3-units-1000-b.h5
67191/67191 [==============================] - 35s 515us/sample - loss: 0.4341 - mean_absolute_error: 0.7928 - val_loss: 0.4351 - val_mean_absolute_error: 0.7942
Epoch 3/50
67136/67191 [============================>.] - ETA: 0s - loss: 0.4200 - mean_absolute_error: 0.77